# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data_by_customer = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
display(data_by_customer.head())

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
matrix = pd.pivot_table(data_by_customer, index='ProductName', columns ='CustomerID', fill_value= 0)
display(matrix.head())

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube                0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp        0     0     0     0     0     0   
Appetizer - Mushroom Tart                0     0     0     0     0     0   
Appetizer - Sausage Rolls                0     0     0     0     0     0   
Apricots - Dried                         1     0     0     0     1     0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube             0     1     0     0  ...     0    25   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0  ...    25    25   
Appetizer - Mushroom Tart             0     1     0     0  ...    25     0   
Appetizer - Sausage Rolls             0     0     0     0  ...     0     0   
Apricots - Dried                      0     0     0     0  ...     0    25   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0    25   
Appetizer - Sausage Rolls             0     0    25    25    25     0    25   
Apricots - Dried                      0     0     0     0     0     0     0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube             0  
Appetizer - Mini Egg Roll, Shrimp     0  
Appetizer - Mushroom Tart             0  
Appetizer - Sausage Rolls             0  
Apricots - Dried                      0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [31]:
# distances = pdist(matrix.T, 'euclidean') 
# Transponemos porque nos interesan las similitudes entre clientes (columnas)
# squareform = squareform(distances)

similarity = pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'euclidean'))),
                          index=matrix.columns.get_level_values(level=1), 
                          columns=matrix.columns.get_level_values(level=1)) # Deshaciendo el multi_index

In [32]:
similarity.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [40]:
# List of the top 5 most similar customers for the customer with CustomerID 33:
similarity[33].sort_values(ascending=False)[1:6]

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

In [50]:
[e for e in similarity[33].sort_values(ascending=False)[1:6].index]

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [76]:
# Voy a concatenar dataframes, porque como he reseteado el índice, no puedo usar loc... :P 

for e in similarity[33].sort_values(ascending=False)[1:6].index:
    df_aux = data_by_customer[data_by_customer['CustomerID'] == e]
    if e == similarity[33].sort_values(ascending=False)[1:6].index[0]:
        df = df_aux
    else:
        df = pd.concat([df,df_aux])
df.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


In [77]:
set(df['CustomerID'])

{264, 2503, 3305, 3317, 3535}

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [79]:
products_purchased = df.groupby(['ProductName'])['Quantity'].sum().reset_index().sort_values('Quantity', ascending=False)
products_purchased.head()

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [141]:
ProductNameList = matrix['Quantity'].index
QuantityList = matrix['Quantity'][33]
df_33 = pd.DataFrame(ProductNameList) 
df_33['Quantity'] = list(QuantityList)
df_33.head()

,ProductName,Quantity
0,Anchovy Paste - 56 G Tube,0
1,"Appetizer - Mini Egg Roll, Shrimp",0
2,Appetizer - Mushroom Tart,0
3,Appetizer - Sausage Rolls,0
4,Apricots - Dried,1


In [150]:
mergedDF = products_purchased.merge(df_33, on='ProductName')
mergedDF = mergedDF.rename(columns={'Quantity_x': 'Quantity_TOP5', 'Quantity_y': 'Quantity_selected_customer'})
mergedDF.head()

,ProductName,Quantity_TOP5,Quantity_selected_customer
0,Butter - Unsalted,3,0
1,Wine - Ej Gallo Sierra Valley,3,0
2,Towels - Paper / Kraft,3,1
3,Soup - Campbells Bean Medley,3,0
4,Wine - Blue Nun Qualitatswein,3,0


In [163]:
# Filter for records where the chosen customer has not purchased the product.
filteredDF = mergedDF[mergedDF['Quantity_selected_customer']==0]
# Show the top 5 results:
display(filteredDF.head())
print('top 5 results:', ', '.join(list(filteredDF.ProductName[0:5])))

,ProductName,Quantity_TOP5,Quantity_selected_customer
0,Butter - Unsalted,3,0
1,Wine - Ej Gallo Sierra Valley,3,0
3,Soup - Campbells Bean Medley,3,0
4,Wine - Blue Nun Qualitatswein,3,0
6,Chicken - Soup Base,2,0


top 5 results: Butter - Unsalted, Wine - Ej Gallo Sierra Valley, Soup - Campbells Bean Medley, Wine - Blue Nun Qualitatswein, Chicken - Soup Base


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
recommendations = {}
uniqueCustomerIDs = list(set(data.CustomerID))

In [173]:
len(uniqueCustomerIDs)

1000

In [170]:
for ID in uniqueCustomerIDs:
    
    for e in similarity[ID].sort_values(ascending=False)[1:6].index:
        df_aux = data_by_customer[data_by_customer['CustomerID'] == e]
        if e == similarity[ID].sort_values(ascending=False)[1:6].index[0]:
            df = df_aux
        else:
            df = pd.concat([df,df_aux])
            
    products_purchased = df.groupby(
        ['ProductName'])['Quantity'].sum().reset_index().sort_values('Quantity', ascending=False)
    
    ProductNameList = matrix['Quantity'].index
    QuantityList = matrix['Quantity'][ID]
    df_cust = pd.DataFrame(ProductNameList)
    df_cust['Quantity'] = list(QuantityList)

    mergedDF = products_purchased.merge(df_cust, on='ProductName')
    mergedDF = mergedDF.rename(columns={'Quantity_x': 'Quantity_TOP5', 'Quantity_y': 'Quantity_selected_customer'})
    
    filteredDF = mergedDF[mergedDF['Quantity_selected_customer']==0]
    
    recommendations[ID] = list(filteredDF.ProductName[0:5])

In [172]:
len(recommendations)

1000

##  Step 9: Store the results in a Pandas data frame. The data frame should have a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [204]:
CustomerID = [item for sublist in [[e]*5 for e in recommendations.keys()] for item in sublist]
df_recommendations = pd.DataFrame(CustomerID)
df_recommendations['recommendations'] = [item for sublist in recommendations.values() for item in sublist]
df_recommendations = df_recommendations.rename(columns={0: 'CustomerID'})
df_recommendations.head()

,CustomerID,recommendations
0,83973,Cocoa Butter
1,83973,Veal - Slab Bacon
2,83973,Bandage - Fexible 1x3
3,83973,Ezy Change Mophandle
4,83973,Chicken - Wieners


In [ ]:
# Más fácil con melt... :D
df_recommendations_2 = pd.DataFrame(recommendations)
columns = df_recommendations_2.columns
pd.melt(df_recommendations_2, value_vars= columns)

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [206]:
similarity = pd.DataFrame(1/(1 + squareform(pdist(matrix.T, 'cosine'))),
                          index=matrix.columns.get_level_values(level=1), 
                          columns=matrix.columns.get_level_values(level=1)) # Deshaciendo el multi_index

recommendations = {}
uniqueCustomerIDs = list(set(data.CustomerID))

for ID in uniqueCustomerIDs:
    
    for e in similarity[ID].sort_values(ascending=False)[1:6].index:
        df_aux = data_by_customer[data_by_customer['CustomerID'] == e]
        if e == similarity[ID].sort_values(ascending=False)[1:6].index[0]:
            df = df_aux
        else:
            df = pd.concat([df,df_aux])
            
    products_purchased = df.groupby(
        ['ProductName'])['Quantity'].sum().reset_index().sort_values('Quantity', ascending=False)
    
    ProductNameList = matrix['Quantity'].index
    QuantityList = matrix['Quantity'][ID]
    df_cust = pd.DataFrame(ProductNameList)
    df_cust['Quantity'] = list(QuantityList)

    mergedDF = products_purchased.merge(df_cust, on='ProductName')
    mergedDF = mergedDF.rename(columns={'Quantity_x': 'Quantity_TOP5', 'Quantity_y': 'Quantity_selected_customer'})
    
    filteredDF = mergedDF[mergedDF['Quantity_selected_customer']==0]
    
    recommendations[ID] = list(filteredDF.ProductName[0:5])

CustomerID = [item for sublist in [[e]*5 for e in recommendations.keys()] for item in sublist]
df_recommendations = pd.DataFrame(CustomerID)
df_recommendations['recommendations'] = [item for sublist in recommendations.values() for item in sublist]
df_recommendations = df_recommendations.rename(columns={0: 'CustomerID'})
df_recommendations.head()    

,CustomerID,recommendations
0,83973,Pastry - Choclate Baked
1,83973,Flour - Pastry
2,83973,Puree - Passion Fruit
3,83973,Lettuce - Spring Mix
4,83973,Broom - Corn


In [209]:
# Más fácil con melt... :D
df_recommendations_2 = pd.DataFrame(recommendations)
columns = df_recommendations_2.columns
df_recommendations_2 = pd.melt(df_recommendations_2, value_vars= columns)
df_recommendations_2.head()

,variable,value
0,83973,Pastry - Choclate Baked
1,83973,Flour - Pastry
2,83973,Puree - Passion Fruit
3,83973,Lettuce - Spring Mix
4,83973,Broom - Corn


In [ ]:
# Obtenemos recomendaciones muy distintas, porque 'cosine' calcula distancias teniendo en cuenta sólo la dirección
# de los vectores (ángulo), no sus longitudes.